**INSTALL DUCKDB AND CONFIGURE IT**

In [1]:
%pip install duckdb duckdb-engine jupysql
%pip install leafmap

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import duckdb
import pandas as pd
import leafmap
import os
import zipfile

In [3]:
# Import jupysql Jupyter extension to create SQL cells
%load_ext sql

%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

In [4]:
%%sql
duckdb:///:memory:

INSTALL spatial;
LOAD spatial;

INSTALL httpfs;
LOAD httpfs;

,Success


**LOAD DATA**

In [5]:
%%sql
DROP TABLE IF EXISTS crane;
CREATE TABLE crane AS SELECT * FROM 'white_naped_crane_data/White-naped crane Mongolia WSCC.csv';

,Success


## Get main data from original dataset

In [6]:
%%sql
-- get name of columns, type of variables and more info
SELECT * FROM (DESCRIBE crane);


,column_name,column_type,null,key,default,extra
0,event-id,BIGINT,YES,None,None,None
1,visible,BOOLEAN,YES,None,None,None
2,timestamp,TIMESTAMP,YES,None,None,None
3,location-long,DOUBLE,YES,None,None,None
4,location-lat,DOUBLE,YES,None,None,None
5,gps:fix-type,BIGINT,YES,None,None,None
6,gps:hdop,DOUBLE,YES,None,None,None
7,ground-speed,DOUBLE,YES,None,None,None
8,heading,DOUBLE,YES,None,None,None
9,height-above-msl,DOUBLE,YES,None,None,None


MAIN OPERATIONS AVAILABLE

In [7]:
%%sql

SELECT COUNT(*) FROM crane;

,count_star()
0,234390


In [8]:
%%sql

SELECT * FROM crane LIMIT 10;

,event-id,visible,timestamp,location-long,location-lat,gps:fix-type,gps:hdop,ground-speed,heading,height-above-msl,manually-marked-outlier,tag-voltage,sensor-type,individual-taxon-canonical-name,tag-local-identifier,individual-local-identifier,study-name
0,2356091477,True,2014-08-09 08:28:09,110.218422,48.126228,3,3.5,12.346667,NaN,1215.0,None,3.66,gps,Grus vipio,8.946080e+19,27233186,White-naped crane Mongolia WSCC
1,2356091577,True,2014-08-09 08:59:09,110.335327,48.204704,3,1.7,12.346667,NaN,1241.0,None,3.65,gps,Grus vipio,8.946080e+19,27233186,White-naped crane Mongolia WSCC
2,2356091532,True,2014-08-09 09:27:39,110.354172,48.295742,3,4.5,0.514444,NaN,909.0,None,3.66,gps,Grus vipio,8.946080e+19,27233186,White-naped crane Mongolia WSCC
3,2356091482,True,2014-08-09 09:58:09,110.353500,48.298092,3,2.5,0.000000,NaN,1010.0,None,3.66,gps,Grus vipio,8.946080e+19,27233186,White-naped crane Mongolia WSCC
4,2356091504,True,2014-08-09 10:29:09,110.354248,48.297897,3,2.9,0.000000,NaN,1133.0,None,3.65,gps,Grus vipio,8.946080e+19,27233186,White-naped crane Mongolia WSCC
5,2356091478,True,2014-08-09 10:58:40,110.354210,48.297924,3,1.4,0.000000,NaN,1132.0,None,3.65,gps,Grus vipio,8.946080e+19,27233186,White-naped crane Mongolia WSCC
6,2356091533,True,2014-08-09 11:29:09,110.354233,48.297924,3,3.8,0.000000,NaN,1133.0,None,3.65,gps,Grus vipio,8.946080e+19,27233186,White-naped crane Mongolia WSCC
7,2356091578,True,2014-08-09 11:43:09,110.354218,48.297981,3,1.7,0.000000,NaN,1133.0,None,3.65,gps,Grus vipio,8.946080e+19,27233186,White-naped crane Mongolia WSCC
8,2356091483,True,2014-08-09 12:14:09,110.354240,48.297970,3,1.2,0.000000,NaN,1133.0,None,3.64,gps,Grus vipio,8.946080e+19,27233186,White-naped crane Mongolia WSCC
9,2356091505,True,2014-08-09 12:44:03,110.354179,48.297890,3,1.5,0.000000,NaN,1139.0,None,3.63,gps,Grus vipio,8.946080e+19,27233186,White-naped crane Mongolia WSCC


In [9]:
%%sql

SELECT MIN(timestamp), MAX(timestamp) FROM crane;


,"min(""timestamp"")","max(""timestamp"")"
0,2013-08-13 00:29:39,2021-04-10 23:02:45


In [10]:
%%sql

SELECT DISTINCT "individual-local-identifier" FROM crane
LIMIT 14;

,individual-local-identifier
0,27234135
1,27230893
2,27233947
3,52409530
4,27233186
5,52409548
6,27231081


In [11]:
%%sql

SELECT COUNT(DISTINCT "individual-local-identifier") FROM crane AS number_of_cranes;

,"count(DISTINCT ""individual-local-identifier"")"
0,7


In [12]:
%%sql

SELECT "individual-local-identifier", COUNT(*) AS count
FROM crane GROUP BY "individual-local-identifier";


,individual-local-identifier,count
0,27233947,79194
1,52409530,11624
2,27234135,41513
3,27230893,13747
4,27233186,17657
5,52409548,86
6,27231081,70569


### LOAD DATA USING PANDAS, CLEAN IT AND VISUALIZE IT

In [13]:
df = pd.read_csv('white_naped_crane_data/White-naped crane Mongolia WSCC.csv')

In [14]:
df.head()

,event-id,visible,timestamp,location-long,location-lat,gps:fix-type,gps:hdop,ground-speed,heading,height-above-msl,manually-marked-outlier,tag-voltage,sensor-type,individual-taxon-canonical-name,tag-local-identifier,individual-local-identifier,study-name
0,2356091477,True,2014-08-09 08:28:09.000,110.218422,48.126228,3,3.5,12.346667,NaN,1215.0,NaN,3.66,gps,Grus vipio,89460800120027233186,27233186,White-naped crane Mongolia WSCC
1,2356091577,True,2014-08-09 08:59:09.000,110.335327,48.204704,3,1.7,12.346667,NaN,1241.0,NaN,3.65,gps,Grus vipio,89460800120027233186,27233186,White-naped crane Mongolia WSCC
2,2356091532,True,2014-08-09 09:27:39.000,110.354172,48.295742,3,4.5,0.514444,NaN,909.0,NaN,3.66,gps,Grus vipio,89460800120027233186,27233186,White-naped crane Mongolia WSCC
3,2356091482,True,2014-08-09 09:58:09.000,110.353500,48.298092,3,2.5,0.000000,NaN,1010.0,NaN,3.66,gps,Grus vipio,89460800120027233186,27233186,White-naped crane Mongolia WSCC
4,2356091504,True,2014-08-09 10:29:09.000,110.354248,48.297897,3,2.9,0.000000,NaN,1133.0,NaN,3.65,gps,Grus vipio,89460800120027233186,27233186,White-naped crane Mongolia WSCC


In [ ]:
df_clean = df[df['manually-marked-outlier'] != True]


In [18]:
import geopandas as gpd
from shapely.geometry import Point

# Create a GeoDataFrame from the DataFrame
geometry = [Point(xy) for xy in zip(df_clean['location-long'], df_clean['location-lat'])]
gdf = gpd.GeoDataFrame(df_clean, geometry=geometry)

# Set the coordinate reference system (CRS) to WGS84 (EPSG:4326)
gdf.set_crs(epsg=4326, inplace=True)

# Drop rows with NaN values in 'location-lat' and 'location-long' columns
df_clean = gdf.dropna(subset=['location-lat', 'location-long'])


In [ ]:
import folium
import matplotlib.pyplot as plt
from matplotlib.colors import rgb2hex


# Create a map centered around the mean location
mean_lat = df_clean['location-lat'].mean()
mean_lon = df_clean['location-long'].mean()
m = folium.Map(location=[mean_lat, mean_lon], zoom_start=5)  # Adjusted zoom level

# Group the data by individual-local-identifier
grouped = df_clean.groupby('individual-local-identifier')

# Generate a color map
colors = plt.cm.get_cmap('tab20', len(grouped))
color_map = {name: colors(i) for i, name in enumerate(grouped.groups.keys())}

# Add polylines and circle markers for each individual
for name, group in grouped:
    color = rgb2hex(color_map[name])
    locations = group.sort_values('timestamp')[['location-lat', 'location-long']].values.tolist()
    folium.PolyLine(locations, color=color, weight=2.5, opacity=1).add_to(m)
    for idx, row in group.iterrows():
        folium.CircleMarker(
            location=[row['location-lat'], row['location-long']],
            radius=1,
            color=color,
            fill=True,
            fill_color=color,
        ).add_to(m)

# Add a different tile layer
folium.TileLayer('CartoDB positron', attr='Map data © OpenStreetMap contributors').add_to(m)

# Add layer control
folium.LayerControl().add_to(m)

# Display the map
m